<a href="https://colab.research.google.com/github/seenu-g/Experiments/blob/master/chap12/Trial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U git+https://github.com/albu/albumentations > /dev/null && echo "Albumentation library is successfully installed!"
!pip install --no-cache-dir git+https://github.com/NVIDIA/apex > /dev/null && echo "Apex library is successfully installed!"!

  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-b2mr40rh
Albumentation library is successfully installed!
  Running command git clone -q https://github.com/NVIDIA/apex /tmp/pip-req-build-o_15ea3q
  Running command git submodule update --init --recursive -q
Apex library is successfully installed!!


In [3]:
# mount gdrive and unzip data
from google.colab import drive
import sys
%load_ext autoreload
%autoreload 2
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
import sys
import os
folder = '/content/gdrive/My Drive/school_of_ai/chap12/'
os.chdir(folder)
print(os.getcwd()) 
for items in os.listdir(folder):
   print(items)
sys.path.append(folder)

/content/gdrive/My Drive/school_of_ai/chap12
template
tiny-imagenet-200.zip
tiny-imagenet-200
Trial1.ipynb
Trial2.ipynb


In [0]:
import os
import requests
from zipfile import ZipFile 
from io import StringIO
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
from PIL import Image
from torchvision import transforms, utils

In [0]:
def download_images(url,fileName):
    if(os.path.isfile(fileName)==False) :
          print ('Downloading ' + url )
          try:
                r = requests.get(url, stream=True)
                with open(fileName,'wb') as f: 
                  f.write(r.content) 
          except :
                print ("Exception to download file",fileName)
                return False
    else:
        print (fileName, " already exists in ",os. getcwd()) 
    
def extract_zip_file(download_folder,fileName):
    
    if (os.path.isdir(download_folder)==False):
        print(download_folder," does not exist")
        return    
    if(os.path.isfile(fileName)==False) :
        print(fileName," from which to extract does not exist")
        return
    
    os.chdir(download_folder)
    extracted_folder = download_folder + fileName.split('.')[0]
    
    if (os.path.isdir(extracted_folder)==True):
        print ('Images seems to be downloaded and extracted already... to',extracted_folder)
        return

    with ZipFile(fileName, 'r') as zip: 
         try:
              #zip.printdir() 
              print('Extracting file ' + fileName + ' now... to ',extracted_folder) 
              zip.extractall() 
              print('File Extraction Done!') 
              return 
         except:
              print ("Exception to Unzip file",fileName)
              return 

In [0]:
def read_val_boxes(fileName) :
       val_boxes_data = pd.read_csv(fileName, delim_whitespace=True, header=None, names=['Images', 'Class', '1', '2', '3', '4'])
       return val_boxes_data
  
def read_val_annotations(fileName) :
        val_annotation_data = pd.read_csv(fileName, delim_whitespace=True, header=None, names=['File', 'Class', '1', '2', '3', '4'])
        return val_annotation_data
    
def get_label_from_name(data, name):
    for row in data.iterrows():       
        if (row['File'] == name):
            return row['Class']  

def get_classes(download_folder):
    classes = []
    wnids = open(os.path.join(download_folder,"wnids.txt"), "r")
    for line in wnids:
        classes.append(line.strip())
    return classes

def load_train_data(download_folder,classes):
    train_data = []
    train_target = []
    train_folder = os.path.join(download_folder,'train')
    for classKey in os.listdir(train_folder):
        class_folder = os.path.join(train_folder, classKey)
        boxes_filepath = os.path.join(class_folder , classKey+'_boxes.txt')
        boxes_records = read_val_boxes(boxes_filepath)
        images_folder = class_folder + '/images/'
        for image_file in os.listdir(images_folder):
            a = Image.open(images_folder+image_file)
            npimg = np.asarray(a)
            if(len(npimg.shape) ==2):
              npimg = np.repeat(npimg[:, :, np.newaxis], 3, axis=2)
            train_data.append(npimg)  
            train_target.append(classes.index(classKey)) 
    return train_data,train_target

def load_val_data(download_folder):
    val_data = []
    val_labels = []
    val_folder = os.path.join(download_folder,'val')
    annotations_filepath = os.path.join(val_folder , 'val_annotations.txt')
    annotations_data = read_val_annotations(annotations_filepath)
    images_folder = val_folder + '/images/'
    for image_file in os.listdir(images_folder):
        a = Image.open(images_folder+image_file)
        npimg = np.asarray(a)
        if(len(npimg.shape) ==2):
              npimg = np.repeat(npimg[:, :, np.newaxis], 3, axis=2)
        val_data.append(npimg)  
        val_labels.append(get_label_from_name(annotations_data, image_file))
    return val_data,val_labels


class TintImageNetTrainDataset(Dataset):
    
    def __init__(self, data_root,classes):
        self.classes = classes
        self.train_data, self.train_target = load_train_data(data_root,classes)

    def __len__(self):
        return len(self.train_data)

    def __getitem__(self, idx):
        data = self.train_data[idx]
        target = self.train_target[idx]
        img = data
        self.transform =  transforms.Compose(
            [   
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        img = self.transform(img)
        return img,target
    
class TintImageNetValDataset(Dataset):
 
    def __init__(self, data_root, classes):
        
        self.classes = classes
        self.val_data, self.val_labels = load_val_data(data_root)

    def __len__(self):
        return len(self.val_data)

    def __getitem__(self, idx):
        data = self.val_data[idx]
        target = self.val_labels[idx]
        img = data
        self.transform =  transforms.Compose(
            [   
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        img = self.transform(img)
        return img,target

In [0]:
IMAGES_URL = 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'
fileName = "tiny-imagenet-200.zip"

download_images(IMAGES_URL)
extract_zipfile(folder,fileName)

In [8]:
download_folder = folder + 'tiny-imagenet-200/'
if (os.path.isdir(download_folder)==False):
    print(download_folder," does not exist")
else :    
    classes = get_classes(download_folder)
    print(classes)

['n02124075', 'n04067472', 'n04540053', 'n04099969', 'n07749582', 'n01641577', 'n02802426', 'n09246464', 'n07920052', 'n03970156', 'n03891332', 'n02106662', 'n03201208', 'n02279972', 'n02132136', 'n04146614', 'n07873807', 'n02364673', 'n04507155', 'n03854065', 'n03838899', 'n03733131', 'n01443537', 'n07875152', 'n03544143', 'n09428293', 'n03085013', 'n02437312', 'n07614500', 'n03804744', 'n04265275', 'n02963159', 'n02486410', 'n01944390', 'n09256479', 'n02058221', 'n04275548', 'n02321529', 'n02769748', 'n02099712', 'n07695742', 'n02056570', 'n02281406', 'n01774750', 'n02509815', 'n03983396', 'n07753592', 'n04254777', 'n02233338', 'n04008634', 'n02823428', 'n02236044', 'n03393912', 'n07583066', 'n04074963', 'n01629819', 'n09332890', 'n02481823', 'n03902125', 'n03404251', 'n09193705', 'n03637318', 'n04456115', 'n02666196', 'n03796401', 'n02795169', 'n02123045', 'n01855672', 'n01882714', 'n02917067', 'n02988304', 'n04398044', 'n02843684', 'n02423022', 'n02669723', 'n04465501', 'n02165456'

In [9]:
print(download_folder)

/content/gdrive/My Drive/school_of_ai/chap12/tiny-imagenet-200/


In [0]:
train_data, train_target = load_train_data(download_folder, classes)

In [0]:
   val_data, val_target = load_val_data(download_folder)